In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from scipy.stats import linregress
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [73]:
!ls

CIR-model.ipynb IR-data.xlsx    README.md


In [92]:
ir = pd.read_excel('IR-data.xlsx')
ir['delta'] = ir['Interest rate'].diff()
# ir.loc[:, ['Interest rate', 'delta']] = ir.loc[:, ['Interest rate', 'delta']].multiply(0.01)
ir['ir_sqrt'] = np.sqrt(ir['Interest rate'].shift(1))
ir['const'] = 1 / np.sqrt(ir['Interest rate'].shift(1))
ir = ir.rename(columns={'Interest rate': 'interest_rate'})
# ir = ir.bfill()
ir = ir.dropna()
ir

,Date,interest_rate,delta,ir_sqrt,const
1,2011-01-06,4.13,-0.07,2.049390,0.487950
2,2011-01-13,4.08,-0.05,2.032240,0.492068
3,2011-01-20,4.05,-0.03,2.019901,0.495074
4,2011-01-27,4.09,0.04,2.012461,0.496904
5,2011-02-03,4.08,-0.01,2.022375,0.494468
...,...,...,...,...,...
582,2022-02-24,3.14,-0.01,1.774824,0.563436
583,2022-03-03,3.01,-0.13,1.772005,0.564333
584,2022-03-10,3.09,0.08,1.734935,0.576390
585,2022-03-17,3.39,0.30,1.757840,0.568880


In [94]:
train = ir.loc[ir['Date'] < '2021-01-01', :]
test = ir.loc[ir['Date'] >= '2021-01-01', :]

x_train = train.loc[:, ['const', 'ir_sqrt']]
y_train = train.loc[:, 'delta'] / train.loc[:, 'ir_sqrt']

x_test = test.loc[:, ['const', 'ir_sqrt']]
y_test = test.loc[:, 'delta'] / test.loc[:, 'ir_sqrt']

x_train

,const,ir_sqrt
1,0.487950,2.049390
2,0.492068,2.032240
3,0.495074,2.019901
4,0.496904,2.012461
5,0.494468,2.022375
...,...,...
518,0.662266,1.509967
519,0.665190,1.503330
520,0.665190,1.503330
521,0.672673,1.486607


In [95]:
linreg = smf.ols('delta ~ const + ir_sqrt - 1', data=train).fit()
linreg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  delta   R-squared (uncentered):                   0.007
Model:                            OLS   Adj. R-squared (uncentered):              0.004
Method:                 Least Squares   F-statistic:                              1.947
Date:                Sun, 25 Aug 2024   Prob (F-statistic):                       0.144
Time:                        21:04:32   Log-Likelihood:                          708.73
No. Observations:                 522   AIC:                                     -1413.
Df Residuals:                     520   BIC:                                     -1405.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0446      0.035      1.260      0.208      -0.025       0.114
ir_sqrt       -0.0161      0.011     -1.457      0.146      -0.038       0.006
==============================================================================
Omnibus:                      173.722   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1220.354
Skew:                           1.267   Prob(JB):                    1.01e-265
Kurtosis:                      10.049   Cond. No.                         25.6
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
a_value = -1 * linreg.params[1]
b_value = linreg.params[0] / a_value
ir_t = train.loc[len(train)-1:, 'interest_rate'].values[0] # last value of interest rate on training set
sigma = linreg.mse_resid # MSE for Residuals / Variance for the model
print(f' a: {a_value} \n b: {b_value} \n σ: {sigma} \n last training data interest rate: {ir_t}')

 a: 0.01611891912965059 
 b: 2.7698510266745124 
 σ: 0.0038895286133488393 
 last training data interest rate: 2.19


In [103]:
# forecasting with Vasicek model formula
cir_pred = []
for day in tqdm(range(len(test))):
    new_ir = ir_t * np.exp(-1 * a_value * day) + b_value * (1 - np.exp(-1 * a_value * day))
    new_var = (1 - np.exp(-2 * a_value * day)) * sigma / (2 * a_value) + ((np.exp(-1 * a_value * day)) - (np.exp(-2 * a_value * day))) * sigma * ir_t / a_value
    cir_pred.append([new_ir, new_var])

cir_pred_df = pd.DataFrame(cir_pred, columns=['interest_rate', 'Variance'])
cir_pred_df['Date'] = test['Date'].values
cir_pred_df = cir_pred_df[['Date', 'interest_rate', 'Variance']]
cir_pred_df

100%|██████████| 64/64 [00:00<00:00, 85789.54it/s]


,Date,interest_rate,Variance
0,2021-01-07,2.190000,0.000000
1,2021-01-14,2.199272,0.012142
2,2021-01-21,2.208395,0.023766
3,2021-01-28,2.217373,0.034891
4,2021-02-04,2.226207,0.045533
...,...,...,...
59,2022-02-24,2.545827,0.227929
60,2022-03-03,2.549409,0.227738
61,2022-03-10,2.552934,0.227502
62,2022-03-17,2.556402,0.227223


In [104]:
# long run forecast
ir_long = b_value
vol_long = sigma / (2 * a_value)
print(f'Long run interest rate: {ir_long} \nLong run variance: {vol_long}')

Long run interest rate: 2.7698510266745124 
Long run variance: 0.1206510368984385


In [105]:
# OLS Prediction / forecasting with linear regression model
pred = linreg.predict(x_test)
linreg_mse = np.mean((pred - y_test) ** 2)
pred, linreg_mse, linreg.rsquared

(523    0.006564
 524    0.006689
 525    0.005827
 526    0.006070
 527    0.006193
          ...   
 582   -0.003453
 583   -0.003367
 584   -0.002231
 585   -0.002936
 586   -0.005429
 Length: 64, dtype: float64,
 0.002660956669065971,
 0.007432458683125143)

In [106]:
ir4ols = ir.loc[len(train) - 1:, ['Date', 'interest_rate']]
ir4ols['interest_rate'] = ir4ols['interest_rate'].shift(1)
ir4ols.loc[:, 'interest_rate'] = ir4ols.loc[:, 'interest_rate'] + pred.values[1]
ir4ols = ir4ols.dropna()
ir4ols

,Date,interest_rate
522,2020-12-31,2.196689
523,2021-01-07,2.176689
524,2021-01-14,2.166689
525,2021-01-21,2.236689
526,2021-01-28,2.216689
...,...,...
582,2022-02-24,3.156689
583,2022-03-03,3.146689
584,2022-03-10,3.016689
585,2022-03-17,3.096689


In [107]:
ols_cir = pd.merge(ir4ols, cir_pred_df, on='Date', suffixes=('_ols', '_cir'))
ols_cir

,Date,interest_rate_ols,interest_rate_cir,Variance
0,2021-01-07,2.176689,2.190000,0.000000
1,2021-01-14,2.166689,2.199272,0.012142
2,2021-01-21,2.236689,2.208395,0.023766
3,2021-01-28,2.216689,2.217373,0.034891
4,2021-02-04,2.206689,2.226207,0.045533
...,...,...,...,...
59,2022-02-24,3.156689,2.545827,0.227929
60,2022-03-03,3.146689,2.549409,0.227738
61,2022-03-10,3.016689,2.552934,0.227502
62,2022-03-17,3.096689,2.556402,0.227223


In [108]:
mse = np.mean((ols_cir['interest_rate_ols'] - ols_cir['interest_rate_cir']) ** 2)
mse

0.057915517305989865